*참고: <https://shinminyong.tistory.com/2?category=835486>

In [102]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request 
import re
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

In [103]:
browser =  Chrome(executable_path='E:/chromedriver_win32 (1)/chromedriver.exe')

In [104]:
base_url = 'http://www.youtube.com'
browser.get(base_url)
browser.maximize_window()

In [107]:
browser.find_elements_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div/div[1]/div/div[2]/input')[0].click()

In [108]:
browser.find_elements_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div/div[1]/div/div[2]/input')[0].send_keys('dingo freestyle') #원하는 유튜버 입력
browser.find_elements_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div/div[1]/div/div[2]/input')[0].send_keys(Keys.RETURN) #엔터

In [109]:
browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-channel-renderer/div/div[2]/a/div[1]/ytd-channel-name/div/div/yt-formatted-string').click()

In [110]:
browser.find_element_by_xpath('//*[@class="scrollable style-scope paper-tabs"]/paper-tab[2]').click()

In [112]:
body = browser.find_element_by_tag_name('body')

In [120]:
num_of_pagedowns = 40
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [129]:
html0 = browser.page_source

In [130]:
html = BeautifulSoup(html0,'html.parser')

In [131]:
video_list2 = html.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

In [132]:
dingo_url = []
for i in range(len(video_list2)):
    url = base_url + video_list2[i].find('a',{'id':'thumbnail'})['href']
    dingo_url.append(url)

In [133]:
dingo = pd.DataFrame({'name':[],
                     'thumbnail':[],
                     'view':[],
                     'previous_time':[],
                     'video_url':[],
                     'start_date':[],
                     'comment':[],
                     'likes_num':[],
                     'unlikes_num':[]})

In [135]:
for i in range(0,20):
    #video 제목
    name = video_list2[i].find('a',{'id':'video-title'}).text
    
    #섬네일
    thum = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src']
    
    #url 정보
    url = base_url + video_list2[i].find('a',{'id':'thumbnail'})['href']
    
    meta0 = video_list2[i].find('div',{'id':'metadata-line'})
    
    #조회수는 숫자만 추출 가능
    view = meta0.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})[0].text.split()[1]
    
    previous = meta0.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})[1].text
    
    start_url = dingo_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')
    
    time.sleep(30) #로딩시간 설정
    
    num_of_pagedowns = 10
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        num_of_pagedowns -= 1
    
    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')
    
    start_date = html.find('div',{'id':'date'}).text
    
    comment = html.find('h2',{'id':'count'}).find('yt-formatted-string').text
    
    #좋아요수
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text',
                                                'aria-label': re.compile('좋아요')}).text+'개'
    
    unlikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text',
                                                'aria-label': re.compile('싫어요')}).text+'개'
    
    insert_data = pd.DataFrame({'name':[name],
                               'thumbnail':[thum],
                               'view': [view],
                               'previous_time' : [previous],
                               'video_url':[url],
                               'start_date':[start_date],
                               'comment':[comment],
                               'likes_num':[likes_num],
                               'unlikes_num':[unlikes_num]})
    dingo = dingo.append(insert_data)

KeyError: 'src'

In [137]:
dingo = dingo.reset_index(drop=True)

In [140]:
dingo

,name,thumbnail,view,previous_time,video_url,start_date,comment,likes_num,unlikes_num
0,세상에 공개 되면 안 되는 미방분 모음! / [영리한 문제아들] 비하인드 : 미방분...,https://i.ytimg.com/vi/hDRlWn78e6A/hqdefault.j...,8.7만회,16시간 전,http://www.youtube.com/watch?v=hDRlWn78e6A,•2020. 6. 7.,댓글 451개,2.3천개,24개
1,이 집 제육볶음 잘 하네... I [어이~ 오씨~] 오담률 (김농밀) I TEASER,https://i.ytimg.com/vi/-biNhiZx3SU/hqdefault.j...,18만회,3일 전,http://www.youtube.com/watch?v=-biNhiZx3SU,•2020. 6. 5.,댓글 864개,2.6천개,51개
2,나이 많은 사람은 힙합을 싫어한다고?! 트로트랩을 들은 어르신들의 반응은 과연......,https://i.ytimg.com/vi/FgZjW_QMTW8/hqdefault.j...,20만회,4일 전,http://www.youtube.com/watch?v=FgZjW_QMTW8,•2020. 6. 4.,"댓글 1,590개",4.3천개,63개
3,"[4K] MBC 힙합걸Z (브린(Bryn), 하선호(Sandy), 이영지)의 킬링벌...",https://i.ytimg.com/vi/UDex1bH2vbM/hqdefault.j...,122만회,1주 전,http://www.youtube.com/watch?v=UDex1bH2vbM,•최초 공개: 2020. 5. 29.,"댓글 6,339개",2.8만개,710개
4,남초 회사 근무중... 어떤 오빠가 제일 문제일까? | [영리한 문제아들 EP.4]...,https://i.ytimg.com/vi/JLjQMj9uidM/hqdefault.j...,38만회,1주 전,http://www.youtube.com/watch?v=JLjQMj9uidM,•최초 공개: 2020. 5. 28.,"댓글 1,971개",6.6천개,125개
5,Rohann (이로한)의 첫 정규앨범 NEVERLAND✨✨ | [집에 갈래?] 변하...,https://i.ytimg.com/vi/hjzKy20G6Lg/hqdefault.j...,11만회,1주 전,http://www.youtube.com/watch?v=hjzKy20G6Lg,•2020. 5. 27.,댓글 391개,2.4천개,28개
6,"수퍼비, 트월킹이다 질문받는다 | [영리한 문제아들] 비하인드 : 댓글 읽기",https://i.ytimg.com/vi/r_5gsfcH-Wg/hqdefault.j...,25만회,1주 전,http://www.youtube.com/watch?v=r_5gsfcH-Wg,•2020. 5. 26.,"댓글 1,960개",5.4천개,78개
7,(전) 던밀스 (역)🎉🎉 드디어 싹이 보이는 딩고 VMC 콜라보...? / [던밀...,https://i.ytimg.com/vi/TxevvmA4uho/hqdefault.j...,32만회,2주 전,http://www.youtube.com/watch?v=TxevvmA4uho,•2020. 5. 25.,"댓글 1,262개",5.3천개,67개
8,트로트랩 (Melody Ver.) | [Official Lyric Video] 수퍼...,https://i.ytimg.com/vi/NE8rtMcB15A/hqdefault.j...,46만회,2주 전,http://www.youtube.com/watch?v=NE8rtMcB15A,•2020. 5. 23.,"댓글 2,970개",9.1천개,105개
9,내일 발매될 스월비 정규앨범 미리듣기! I [앨범 스포일러] 스월비 (Swervy)...,https://i.ytimg.com/vi/vbKRXir7114/hqdefault.j...,12만회,2주 전,http://www.youtube.com/watch?v=vbKRXir7114,•2020. 5. 22.,댓글 734개,2.4천개,76개


In [146]:
dingo.to_csv("E:/jyeon94.github.io/_posts/dingo.csv",index=False)

In [166]:
p = re.compile(r"(•)(\d+[.]\d+[.]\d+)")

In [167]:
m = p.search("•2020.6.7.")

In [216]:
dingo['start_date'] = dingo['start_date'].replace("최초 공개","",regex=True).replace("•","",regex=True).replace(":","",regex=True).replace("\.$","",regex=True).str.replace(" ","")

In [ ]:
dingo

In [217]:
pd.read_csv("dingo.csv")

,name,thumbnail,view,previous_time,video_url,start_date,comment,likes_num,unlikes_num
0,세상에 공개 되면 안 되는 미방분 모음! / [영리한 문제아들] 비하인드 : 미방분...,https://i.ytimg.com/vi/hDRlWn78e6A/hqdefault.j...,8.7만회,16시간 전,http://www.youtube.com/watch?v=hDRlWn78e6A,•2020. 6. 7.,댓글 451개,2.3천개,24개
1,이 집 제육볶음 잘 하네... I [어이~ 오씨~] 오담률 (김농밀) I TEASER,https://i.ytimg.com/vi/-biNhiZx3SU/hqdefault.j...,18만회,3일 전,http://www.youtube.com/watch?v=-biNhiZx3SU,•2020. 6. 5.,댓글 864개,2.6천개,51개
2,나이 많은 사람은 힙합을 싫어한다고?! 트로트랩을 들은 어르신들의 반응은 과연......,https://i.ytimg.com/vi/FgZjW_QMTW8/hqdefault.j...,20만회,4일 전,http://www.youtube.com/watch?v=FgZjW_QMTW8,•2020. 6. 4.,"댓글 1,590개",4.3천개,63개
3,"[4K] MBC 힙합걸Z (브린(Bryn), 하선호(Sandy), 이영지)의 킬링벌...",https://i.ytimg.com/vi/UDex1bH2vbM/hqdefault.j...,122만회,1주 전,http://www.youtube.com/watch?v=UDex1bH2vbM,•최초 공개: 2020. 5. 29.,"댓글 6,339개",2.8만개,710개
4,남초 회사 근무중... 어떤 오빠가 제일 문제일까? | [영리한 문제아들 EP.4]...,https://i.ytimg.com/vi/JLjQMj9uidM/hqdefault.j...,38만회,1주 전,http://www.youtube.com/watch?v=JLjQMj9uidM,•최초 공개: 2020. 5. 28.,"댓글 1,971개",6.6천개,125개
5,Rohann (이로한)의 첫 정규앨범 NEVERLAND✨✨ | [집에 갈래?] 변하...,https://i.ytimg.com/vi/hjzKy20G6Lg/hqdefault.j...,11만회,1주 전,http://www.youtube.com/watch?v=hjzKy20G6Lg,•2020. 5. 27.,댓글 391개,2.4천개,28개
6,"수퍼비, 트월킹이다 질문받는다 | [영리한 문제아들] 비하인드 : 댓글 읽기",https://i.ytimg.com/vi/r_5gsfcH-Wg/hqdefault.j...,25만회,1주 전,http://www.youtube.com/watch?v=r_5gsfcH-Wg,•2020. 5. 26.,"댓글 1,960개",5.4천개,78개
7,(전) 던밀스 (역)🎉🎉 드디어 싹이 보이는 딩고 VMC 콜라보...? / [던밀...,https://i.ytimg.com/vi/TxevvmA4uho/hqdefault.j...,32만회,2주 전,http://www.youtube.com/watch?v=TxevvmA4uho,•2020. 5. 25.,"댓글 1,262개",5.3천개,67개
8,트로트랩 (Melody Ver.) | [Official Lyric Video] 수퍼...,https://i.ytimg.com/vi/NE8rtMcB15A/hqdefault.j...,46만회,2주 전,http://www.youtube.com/watch?v=NE8rtMcB15A,•2020. 5. 23.,"댓글 2,970개",9.1천개,105개
9,내일 발매될 스월비 정규앨범 미리듣기! I [앨범 스포일러] 스월비 (Swervy)...,https://i.ytimg.com/vi/vbKRXir7114/hqdefault.j...,12만회,2주 전,http://www.youtube.com/watch?v=vbKRXir7114,•2020. 5. 22.,댓글 734개,2.4천개,76개
